# Introduction

This notebook downloads data provided by the HDX team from a google drive folder. The data was captured using an [HXL crawl process](https://github.com/HXLStandard/hdx-hashtag-crawler)

# Setup

1. Install [miniconda](https://docs.conda.io/en/latest/miniconda.html) by selecting the installer that fits your OS version. Once it is installed you may have to restart your terminal (closing your terminal and opening again)
2. In this directory, open terminal
3. `conda env create -f environment.yml`
4. `conda activate hxl-prediction` use this runtime to run this notebook 
5. Set `OPENAI_API_KEY` in `.env`

In [35]:
import sys
import os
import gdown
import tarfile
import pandas as pd
import re
from sklearn.model_selection import train_test_split

from hdx.utilities.easy_logging import setup_logging
from hdx.api.configuration import Configuration
from hdx.data.dataset import Dataset
import hxl
import json
import time

import numpy as np

import openai
from openai import OpenAI

from dotenv import load_dotenv
load_dotenv()


input_options = hxl.input.InputOptions(http_headers={'User-Agent': "DK HXL Analaysis"})

# Google drive location of HDXHashtag crawler data
DATA_GDRIVE="https://drive.google.com/file/d/1BDCuh0WVJWK1-1RMC-77cvh4H2Hep_ry/view?usp=drive_link"
DATA_FILE="hdx-hxl-output.tgz"

# Where to save local data files
LOCAL_DATA_DIR = "./data/hdx-hxl"

# Number of records in data excerpts
DATA_EXCERPT_SIZE = 10

local_data_file = LOCAL_DATA_DIR + "/" + DATA_FILE + '/output/'

pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', 200)

In [2]:
agent_count = 0

def setup_hdx_connection(agent_name):
    try:
        Configuration.create(hdx_site="prod", user_agent=agent_name, hdx_read_only=True)
    except:
        print("Configuration already created, continuing ...")

# Note, if you run this twice you will get a 'Configuration already exists!' error, but it can be ignored 
setup_hdx_connection(f"DK_UserAgent{agent_count}")

# Analysis

## Download HXL crawler data

This data was generated using the [HDX Hashtag Crawler](https://github.com/dividor/hdx-hashtag-crawler) over several days and saved to Google drive.

In [14]:
# Use gdown to download the file
gdown.download(DATA_GDRIVE, local_data_file, quiet=False, fuzzy=True)

tar = tarfile.open(local_data_file)
tar.extractall(LOCAL_DATA_DIR)
tar.close()

Downloading...
From: https://drive.google.com/uc?id=1BDCuh0WVJWK1-1RMC-77cvh4H2Hep_ry
To: /Users/matthewharris/Desktop/git/hxl_metadata_prediction/data/hdx-hxl/hdx-hxl-output.tgz
100%|██████████| 19.3M/19.3M [00:01<00:00, 10.0MB/s]


## Identify Unique combinations of HXL tags we want for training data

In [135]:
# Open hdx-expanded-hashed-stats.csv in data/hdx-hxl

df = pd.read_csv(LOCAL_DATA_DIR + "/output/hdx-expanded-hashed-stats.csv")

# We'll keep one row per column, Hashtag with Attributes has what we require
df.drop(columns=['Attribute', 'Hashtag'], inplace=True)
df.drop_duplicates(inplace=True)  

# Remove HXL tags row in the metadata (we keep them for actual data)
df = df[1:]

display(df.head())
print(df.shape)

print("Unique data providers ...")
print(len(df["Data provider"].unique()))

print("Unique HDX resource ids ...")
print(len(df["HDX resource id"].unique()))

/var/folders/6b/jxkmd4lx1nl8lc4sdpn0yz500000gn/T/ipykernel_29168/3428572768.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(LOCAL_DATA_DIR + "/output/hdx-expanded-hashed-stats.csv")


,Hashtag with Attributes,Text header,Locations,Data provider,HDX dataset id,HDX resource id,Date created,Unnamed: 9,Hash
1,#affected+hh,Total IDP HH,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,true,0x2cc7fd3129c0d18c
2,#affected+idp+ind,Total IDP IND,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,true,0x2cc7fd3129c0d18c
4,#affected+idp+male,Total IDP Male Ind,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,true,0x2cc7fd3129c0d18c
6,#affected+female+idp,Total IDP Female Ind,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,true,0x2cc7fd3129c0d18c
8,#affected+ind+returnees,Total Returnees,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,true,0x2cc7fd3129c0d18c


(487297, 9)
Unique data providers ...
120
Unique HDX resource ids ...
43074


Let's use the column hash created by the crawler to find unique combinations of tags

In [136]:
hash_count = df.groupby('Hash').size()
display(hash_count)


Hash
0x100556db35012c6b       101
0x102125f4dd16c64     190286
0x10309a2e5e2722ba       509
0x105b36aac3c9192f       693
0x105c6ee3379af31c       595
                       ...  
0xf0a7e4d9104f069         12
0xf93f0051e52a4d          28
0xfa11ad9f842a37d         17
0xfe0e278de9d0a33          1
0xfe8777dcd878424         20
Length: 644, dtype: int64

In [137]:
hash_resources = df.groupby('Hash')['HDX resource id'].nunique().sort_values(ascending=False)

for col in ['HDX resource id', 'HDX dataset id', 'Data provider']:
    hash_resources  = hash_resources .reset_index()
    hash_resources [f"Unique {col}"] = hash_resources ['Hash'].map(df.groupby('Hash')[col].unique())

display(hash_resources )

hash_resources.to_excel("./data/hxl_hash_resources.xlsx", index=False)

,level_0,index,Hash,HDX resource id,Unique HDX resource id,Unique HDX dataset id,Unique Data provider
0,0,0,0x102125f4dd16c64,13858,"[51b2e4ec-aca5-4b97-bbb7-c005175b682e, a6ef8040-3b15-47ae-9973-1dbc113673cf, f579cf0e-5535-4414-897f-2f8c05105180, 66c62464-017b-4aa6-845f-9ec2487acb82, 91e1cb98-353b-487e-a14d-b0eea783da6f, 65f38...","[who-data-for-south-sudan, who-data-for-montenegro, who-data-for-zimbabwe, who-data-for-zambia, who-data-for-yemen, who-data-for-viet-nam, who-data-for-venezuela-bolivarian-republic-of, who-data-f...",[world-health-organization]
1,1,1,0x428a8e37940223d8,9328,"[2e130cdf-c850-4533-b2f3-e961adbec48a, 9e160d82-691d-49a6-979b-0ff0dbb6b7a8, 002501bc-7efb-4335-b672-d045cd76bc5b, 0d0e0fc4-e4f1-49cd-ad30-42cc8dc08b74, f2c1ea93-d241-413b-8140-c009da88d912, 23e36...","[world-bank-combined-indicators-for-zimbabwe, world-bank-trade-indicators-for-zimbabwe, world-bank-external-debt-indicators-for-zimbabwe, world-bank-climate-change-indicators-for-zimbabwe, world-b...",[world-bank-group]
2,2,2,0x31fda1ef985b4a59,3425,"[9c751883-698a-4a2c-9475-ff828e9c11db, 791b69af-df57-4157-96c0-c0d8d308315e, b055f1f7-8cdc-4ff8-a412-9f54d6e56c41, 6ae8568f-449b-4b59-9ad2-79d7df94cd9c, 4f4c7462-6b13-4125-a88a-f8e9ac0c837b, abb79...","[dhs-data-for-sao-tome-and-principe, dhs-data-for-rwanda, dhs-data-for-philippines, dhs-data-for-peru, dhs-data-for-paraguay, dhs-data-for-papua-new-guinea, dhs-data-for-pakistan, dhs-data-for-nig...",[dhs]
3,3,3,0x19598575d3397e19,3232,"[ed7b5bd2-7818-4d7a-9ff0-8ba0d97bf7d5, 8f239e93-76c0-4287-a414-3d17a5e55344, 040efd19-3d71-4e0b-8939-f6b46c465868, 8db41d1c-af8f-4f26-9d40-5b80b1bf72e8, 6482dc75-0e79-40d9-aacf-cdcde3c368a6, c9981...","[dhs-subnational-data-for-sao-tome-and-principe, dhs-subnational-data-for-rwanda, dhs-subnational-data-for-philippines, dhs-subnational-data-for-peru, dhs-subnational-data-for-paraguay, dhs-subnat...",[dhs]
4,4,4,0x16d2b679132fea10,2147,"[295cd9e4-8464-43ee-ad17-47196991a1f7, 34337d16-017d-4d69-834c-a5e0fc21a549, b51b8c0e-494d-488b-98d5-a70fd9451b90, 75076d6a-8d3f-49e3-b4f4-7c889bc82806, 7e0b2b37-73b8-4c69-bb11-6ba954fa0cd9, 08ea0...","[unhcr-population-data-for-world, unhcr-population-data-for-zwe, unhcr-population-data-for-zmb, unhcr-population-data-for-zaf, unhcr-population-data-for-yem, unhcr-population-data-for-wsm, unhcr-p...",[unhcr]
...,...,...,...,...,...,...,...
639,639,639,0x22def8e1b7b0c742,1,[68328f42-9276-423e-80d0-fe89630804ff],[3w-december-2017],[ocha-ethiopia]
640,640,640,0x4b7012601f2de402,1,[63143067-46e0-4fb5-b131-d83ee45122ab],[ethiopia-settlements],[ocha-ethiopia]
641,641,641,0x4aefd35864aaa20a,1,[6ddcfbc9-fa06-4b14-b9a4-ce96d3fae65e],[base-acceso-internet-personas-entre-los-5-y-19-anos-2018],[immap]
642,642,642,0x4acf4e36d67d877,1,[903326f2-b372-4786-9973-87226cb15e41],[people-in-need-2008-2019],[ocha-fts]


In [138]:
# Extract a single resource_id for each hash
resource_ids = hash_resources['Unique HDX resource id'].apply(lambda x: x[0])
print(resource_ids)

0      51b2e4ec-aca5-4b97-bbb7-c005175b682e
1      2e130cdf-c850-4533-b2f3-e961adbec48a
2      9c751883-698a-4a2c-9475-ff828e9c11db
3      ed7b5bd2-7818-4d7a-9ff0-8ba0d97bf7d5
4      295cd9e4-8464-43ee-ad17-47196991a1f7
                       ...                 
639    68328f42-9276-423e-80d0-fe89630804ff
640    63143067-46e0-4fb5-b131-d83ee45122ab
641    6ddcfbc9-fa06-4b14-b9a4-ce96d3fae65e
642    903326f2-b372-4786-9973-87226cb15e41
643    8b0feea6-20e4-45dc-aaae-c8b6fbd5a9f4
Name: Unique HDX resource id, Length: 644, dtype: object


In [140]:
df_subset = df[df['HDX resource id'].isin(resource_ids)]

print("Column data subset to one resource ID per hash ...")
print(df_subset.shape)

print("Unique data providers ...")
print(len(df_subset["Data provider"].unique()))

print("Unique HDX resource ids ...")
print(len(df_subset["HDX resource id"].unique()))

Column data subset to one resource ID per hash ...
(7834, 9)
Unique data providers ...
119
Unique HDX resource ids ...
644


### Download data excerpts

Using our subset of resource_ids for each hash, extract column data excerpts.

In [ ]:
df2 = df_subset.copy()

df2['Data excerpt'] = ''

datasets_resources = df2[['HDX dataset id', 'HDX resource id']].drop_duplicates()
datasets_resources.reset_index(drop=True, inplace=True)

print("For each resource, extract a data excerpt for each column ...")

num_rows = datasets_resources.shape[0]
for index, row in datasets_resources.iterrows():

    if index % 10 == 0:
        print(f"Processing {index} of {num_rows} ({(index/num_rows)*100:.2f}%)")

    dataset_id = row['HDX dataset id']
    resource_id = row['HDX resource id']
    dataset = Dataset.read_from_hdx(dataset_id)
    if dataset is None:
        print(f"Dataset {dataset_id} not found!")
        continue
    resources = dataset.get_resources()
    for resource in resources:
        if resource['id'] == resource_id:
            print(f"    Accessing data for resource {resource_id}, {resource['name']}")
            try:
                url, path = resource.download(LOCAL_DATA_DIR)
                df2.loc[df2['HDX resource id'] == resource_id, 'File'] = path
                df2.loc[df2['HDX resource id'] == resource_id, 'URL'] = url

                with hxl.data(resource['url'], input_options) as source:
                    columns = [column.header for column in source.columns]
                    tags = [column.get_display_tag(sort_attributes=True) for column in source.columns]
                    data = {}
                    rowcount = 0
                    for row in source:
                        if rowcount > DATA_EXCERPT_SIZE:
                            break
                        i = 0
                        for colvalue in row:
                            colname = columns[i]
                            if colname not in data:
                                data[colname] = [colvalue]
                            else:
                                data[colname].append(colvalue)     
                            i += 1
                        rowcount += 1                            

                    for col in columns:
                        if col in data:
                            print(f"       Setting data excerpt for column {col} >> {data[col]} ...")
                            df2.loc[(df2['HDX resource id'] == resource_id) & (df2['Text header'] == col), 'Data excerpt'] = str(data[col])

            except Exception as e:
                print(f"Error accessing data for resource {resource_id}, {resource['name']} ... {e}")

display(df2)
print(df2.shape)
df2.to_csv("./data/hxl_hash_resources_data.csv", index=False)



### Train and test data

In this section we will create train and test datasets for fine tuning.

In [2]:
data = pd.read_csv("./data/hxl_hash_resources_data.csv")
print(data.shape)
display(data)

(7834, 12)


,Hashtag with Attributes,Text header,Locations,Data provider,HDX dataset id,HDX resource id,Date created,Unnamed: 9,Hash,Data excerpt,File,URL
0,#affected+hh,Total IDP HH,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,True,0x2cc7fd3129c0d18c,[319283],./data/hdx-hxl/DRC - Baseline Assessment - M23 Crisis 13 - February 2024.xlsx,https://data.humdata.org/dataset/3554c498-660a-45cb-ada5-86a1fbcd6056/resource/26ecc26f-74e7-46af-b450-8872dca0b63b/download/adc_27jan-12_feb_update_public_v2.xlsx
1,#affected+idp+ind,Total IDP IND,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,True,0x2cc7fd3129c0d18c,[1548732],./data/hdx-hxl/DRC - Baseline Assessment - M23 Crisis 13 - February 2024.xlsx,https://data.humdata.org/dataset/3554c498-660a-45cb-ada5-86a1fbcd6056/resource/26ecc26f-74e7-46af-b450-8872dca0b63b/download/adc_27jan-12_feb_update_public_v2.xlsx
2,#affected+idp+male,Total IDP Male Ind,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,True,0x2cc7fd3129c0d18c,[646805],./data/hdx-hxl/DRC - Baseline Assessment - M23 Crisis 13 - February 2024.xlsx,https://data.humdata.org/dataset/3554c498-660a-45cb-ada5-86a1fbcd6056/resource/26ecc26f-74e7-46af-b450-8872dca0b63b/download/adc_27jan-12_feb_update_public_v2.xlsx
3,#affected+female+idp,Total IDP Female Ind,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,True,0x2cc7fd3129c0d18c,[901927],./data/hdx-hxl/DRC - Baseline Assessment - M23 Crisis 13 - February 2024.xlsx,https://data.humdata.org/dataset/3554c498-660a-45cb-ada5-86a1fbcd6056/resource/26ecc26f-74e7-46af-b450-8872dca0b63b/download/adc_27jan-12_feb_update_public_v2.xlsx
4,#affected+ind+returnees,Total Returnees,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,True,0x2cc7fd3129c0d18c,[587705],./data/hdx-hxl/DRC - Baseline Assessment - M23 Crisis 13 - February 2024.xlsx,https://data.humdata.org/dataset/3554c498-660a-45cb-ada5-86a1fbcd6056/resource/26ecc26f-74e7-46af-b450-8872dca0b63b/download/adc_27jan-12_feb_update_public_v2.xlsx
...,...,...,...,...,...,...,...,...,...,...,...,...
7829,#lat_deg,prevlat,VUT,brcmapsteam,cyclone-pam-path,a8ccd9d2-8328-487a-b04b-ca3f3f2e0ea3,2015-03-16,True,0x1d4a8deeb40f76ce,"['-8.5', '-8.5', '-8.4', '-9.8', '-10.6', '-11.1', '-11', '-11.2', '-11.5', '-11.9', '-12.6']",./data/hdx-hxl/Cyclone Pam Path.google sheet,https://docs.google.com/spreadsheets/d/1xFOPVLCKeVpLtM27loV3_zicG-xswOZk7SD_nAQ217Q/edit?usp=sharing
7830,#lon_deg,prevlon,VUT,brcmapsteam,cyclone-pam-path,a8ccd9d2-8328-487a-b04b-ca3f3f2e0ea3,2015-03-16,True,0x1d4a8deeb40f76ce,"['169.8', '169.8', '170.3', '170.5', '170.3', '170.1', '169.6', '169.7', '169.7', '170.1', '170.2']",./data/hdx-hxl/Cyclone Pam Path.google sheet,https://docs.google.com/spreadsheets/d/1xFOPVLCKeVpLtM27loV3_zicG-xswOZk7SD_nAQ217Q/edit?usp=sharing
7831,#period_date,datelabel,VUT,brcmapsteam,cyclone-pam-path,a8ccd9d2-8328-487a-b04b-ca3f3f2e0ea3,2015-03-16,True,0x1d4a8deeb40f76ce,"['09 Mar', '09 Mar', '10 Mar', '10 Mar', '10 Mar', '11 Mar', '11 Mar', '11 Mar', '11 Mar', '12 Mar', '12 Mar']",./data/hdx-hxl/Cyclone Pam Path.google sheet,https://docs.google.com/spreadsheets/d/1xFOPVLCKeVpLtM27loV3_zicG-xswOZk7SD_nAQ217Q/edit?usp=sharing
7832,#x_time,hours,VUT,brcmapsteam,cyclone-pam-path,a8ccd9d2-8328-487a-b04b-ca3f3f2e0ea3,2015-03-16,True,0x1d4a8deeb40f76ce,"['0', '6', '18', '30', '36', '42', '48', '54', '60', '66', '

#### Data Cleaning

In [3]:
data = data[data['Data excerpt'].notnull()]
data = data[data['Data excerpt'].str.contains(r'[A-Za-z0-9]')]

print(data.shape)

(6737, 12)


In [4]:

HXL_CORE_SCHEMA="https://docs.google.com/spreadsheets/d/1En9FlmM8PrbTWgl3UHPF_MXnJ6ziVZFhBbojSJzBdLI/edit?usp=sharing"

local_data_file = LOCAL_DATA_DIR + "/hxl-core-schema.xlsx"

gdown.download(HXL_CORE_SCHEMA, local_data_file, quiet=False, fuzzy=True)

df= pd.read_excel(local_data_file, sheet_name='Core hashtags')
hashtags_list = df['Hashtag'][1:].tolist()

df= pd.read_excel(local_data_file, sheet_name='Core attributes')
attributes_list = df['Attribute'][1:].tolist()

# Remove rows with disallowed tags or attributes
APPROVED_HXL_SCHEMA = hashtags_list + attributes_list

Downloading...
From (original): https://drive.google.com/uc?id=1En9FlmM8PrbTWgl3UHPF_MXnJ6ziVZFhBbojSJzBdLI
From (redirected): https://docs.google.com/spreadsheets/d/1En9FlmM8PrbTWgl3UHPF_MXnJ6ziVZFhBbojSJzBdLI/export?format=xlsx
To: /Users/matthewharris/Desktop/git/hxl_metadata_prediction/data/hdx-hxl/hxl-core-schema.xlsx
240kB [00:00, 18.6MB/s]


#### Remove disallowed HXL

In [5]:
def filter_for_schema(text):
    #print(f"Tokens before: {text}")
    if " " in text:
        text = text.replace(" ","")

    tokens_raw = text.split("+")
    tokens = [tokens_raw[0]]
    for t in tokens_raw[1:]:
        tokens.append(f"+{t}")

    filtered = []
    for t in tokens:
        if t in APPROVED_HXL_SCHEMA:
            if t not in filtered:
                filtered.append(t)
    filtered = "".join(filtered)

    if len(filtered) > 0 and filtered[0] != '#':
        filtered = ""

    # Add spaces back in
    # filtered = filtered.replace("+", " +")

    #print(f"        After: {filtered}")
    return filtered

def filter_disallowed_hxl(column_data, hxl_col = 'Hashtag with Attributes'):
    print("Before",column_data.shape)
    allowed = []
    disallowed = []
    for index, row in column_data.iterrows():
        if row[hxl_col] == filter_for_schema(row[hxl_col]):
            allowed.append(row)
        else:
            disallowed.append(row)
    allowed = pd.DataFrame(allowed)
    disallowed = pd.DataFrame(disallowed)
    print("After", allowed.shape)
    return allowed, disallowed

data, disallowed = filter_disallowed_hxl(data)
print(data.shape)

display(disallowed)

Before (6737, 12)
After (3368, 12)
(3368, 12)


,Hashtag with Attributes,Text header,Locations,Data provider,HDX dataset id,HDX resource id,Date created,Unnamed: 9,Hash,Data excerpt,File,URL
1,#affected+idp+ind,Total IDP IND,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,True,0x2cc7fd3129c0d18c,[1548732],./data/hdx-hxl/DRC - Baseline Assessment - M23 Crisis 13 - February 2024.xlsx,https://data.humdata.org/dataset/3554c498-660a-45cb-ada5-86a1fbcd6056/resource/26ecc26f-74e7-46af-b450-8872dca0b63b/download/adc_27jan-12_feb_update_public_v2.xlsx
2,#affected+idp+male,Total IDP Male Ind,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,True,0x2cc7fd3129c0d18c,[646805],./data/hdx-hxl/DRC - Baseline Assessment - M23 Crisis 13 - February 2024.xlsx,https://data.humdata.org/dataset/3554c498-660a-45cb-ada5-86a1fbcd6056/resource/26ecc26f-74e7-46af-b450-8872dca0b63b/download/adc_27jan-12_feb_update_public_v2.xlsx
3,#affected+female+idp,Total IDP Female Ind,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,True,0x2cc7fd3129c0d18c,[901927],./data/hdx-hxl/DRC - Baseline Assessment - M23 Crisis 13 - February 2024.xlsx,https://data.humdata.org/dataset/3554c498-660a-45cb-ada5-86a1fbcd6056/resource/26ecc26f-74e7-46af-b450-8872dca0b63b/download/adc_27jan-12_feb_update_public_v2.xlsx
15,#meta+appeal+type,atype,SSD,ifrc,ifrc-appeals-data-for-south-sudan,4110b824-3338-453f-ae5d-89ca80f5b147,2023-03-13,True,0x1d1434ee319a1be,"['0', '0', '0', '1', '0', '0', '2', '0', '0', '1', '0']",./data/hdx-hxl/IFRC Appeals Data for South Sudan.csv,https://data.humdata.org/dataset/db2f8c45-7992-4f23-99f5-e8b4d853b53d/resource/4110b824-3338-453f-ae5d-89ca80f5b147/download/appeals_data_ssd.csv
18,#meta+appeal+id,code,SSD,ifrc,ifrc-appeals-data-for-south-sudan,4110b824-3338-453f-ae5d-89ca80f5b147,2023-03-13,True,0x1d1434ee319a1be,"['MDRSS013', 'MDRSS012', 'MDRSS011', 'MDRSS009', 'MDRSS008', 'MDRSS007', 'MDRSS006', 'MDRSS005', 'MDRSS004', 'MDRSS003', 'MDRSS002']",./data/hdx-hxl/IFRC Appeals Data for South Sudan.csv,https://data.humdata.org/dataset/db2f8c45-7992-4f23-99f5-e8b4d853b53d/resource/4110b824-3338-453f-ae5d-89ca80f5b147/download/appeals_data_ssd.csv
...,...,...,...,...,...,...,...,...,...,...,...,...
7829,#lat_deg,prevlat,VUT,brcmapsteam,cyclone-pam-path,a8ccd9d2-8328-487a-b04b-ca3f3f2e0ea3,2015-03-16,True,0x1d4a8deeb40f76ce,"['-8.5', '-8.5', '-8.4', '-9.8', '-10.6', '-11.1', '-11', '-11.2', '-11.5', '-11.9', '-12.6']",./data/hdx-hxl/Cyclone Pam Path.google sheet,https://docs.google.com/spreadsheets/d/1xFOPVLCKeVpLtM27loV3_zicG-xswOZk7SD_nAQ217Q/edit?usp=sharing
7830,#lon_deg,prevlon,VUT,brcmapsteam,cyclone-pam-path,a8ccd9d2-8328-487a-b04b-ca3f3f2e0ea3,2015-03-16,True,0x1d4a8deeb40f76ce,"['169.8', '169.8', '170.3', '170.5', '170.3', '170.1', '169.6', '169.7', '169.7', '170.1', '170.2']",./data/hdx-hxl/Cyclone Pam Path.google sheet,https://docs.google.com/spreadsheets/d/1xFOPVLCKeVpLtM27loV3_zicG-xswOZk7SD_nAQ217Q/edit?usp=sharing
7831,#period_date,datelabel,VUT,brcmapsteam,cyclone-pam-path,a8ccd9d2-8328-487a-b04b-ca3f3f2e0ea3,2015-03-16,True,0x1d4a8deeb40f76ce,"['09 Mar', '09 Mar', '10 Mar', '10 Mar', '10 Mar', '11 Mar', '11 Mar', '11 Mar', '11 Mar', '12 Mar', '12 Mar']",./data/hdx-hxl/Cyclone Pam Path.google sheet,https://docs.google.com/spreadsheets/d/1xFOPVLCKeVpLtM27loV3_zicG-xswOZk7SD_nAQ217Q/edit?usp=sharing
7832,#x_time,hours,VUT,brcmapsteam,cyclone-pam-path,a8ccd9d2-8328-487a-b04b-ca3f3f2e0ea3,2015-03-16,True,0x1d4a8deeb40f76ce,"['0', '6', '18', '30', '36', '42', '48', '54', '60', '66', '72']",./data/hdx-hxl/Cyclone Pam Path.google sheet,https://docs.google.com/spreadsheets/d/1xFOPVLCKeVpLtM27loV3_zicG-x

#### Plot tags

In [6]:
data['tag'] = data['Hashtag with Attributes'].apply(lambda x: x.split('+')[0])
tag_counts_train = data['tag'].value_counts()

print(tag_counts_train)

tag
#adm1           527
#adm2           477
#affected       380
#country        287
#date           244
#org            232
#adm3           188
#inneed         141
#sector         111
#geo            106
#targeted        77
#loc             67
#activity        67
#status          60
#population      57
#indicator       55
#region          50
#meta            42
#reached         36
#adm4            32
#event           14
#subsector       13
#beneficiary     12
#cause           12
#value           11
#item            10
#severity         9
#output           8
#crisis           6
#currency         4
#service          4
#adm5             4
#contact          4
#access           4
#capacity         3
#impact           3
#description      3
#frequency        2
#group            2
#modality         2
#delivery         1
#operations       1
Name: count, dtype: int64


#### Generate training prompts

####  Split by data provider organization

On HDX, the hierarchy is ...

Organization > datasets > resources > tables

A random train/test split will result in data from files in a dataset being in both train and test, which would pollute the test set with very similar data to training. So we will split by organization

In [8]:
def split_data(column_data, provider_col, test_size=0.2, random_state=42):
    """
    Perform train-test split on datasets, print information, and return X_train and X_test.

    The split is done by organizations, to try and avoid the situation where an org provides
    similar data files. Also, we exclude orgs which are subsidiaries from the test set, eg ocha-*
    as presumably each subsid will provide similar data. The aim is that the test set is new.

    Parameters:
    - column_data (pd.DataFrame): DataFrame containing column data.
    - provider_col (string): Name of column holding data providers.
    - test_size (float): The proportion of the dataset to include in the test split.
    - random_state (int): Seed for random number generation.

    Returns:
    - pd.DataFrame, pd.DataFrame: X_train, X_test
    """
     
    orgs_df = column_data.groupby(provider_col)[provider_col].count().sort_values(ascending=False)
    orgs_df = column_data.groupby(provider_col)[provider_col].count().sort_values(ascending=False).reset_index(name='count')
    all_orgs = orgs_df[provider_col].unique()

    # Split orgs to get 'Parent', eg 'ocha-*' -> 'ocha'
    orgs_df['org_parent'] = orgs_df[provider_col].str.split('-').str[0]

    # Count occurrences of each 'org_parent'
    org_parent_counts = orgs_df['org_parent'].value_counts().reset_index(name='count')

    # Filter to keep only those occurring once
    org_parents_single_occurrence = org_parent_counts[org_parent_counts['count'] == 1]

    # Get the 'org_parent' values that occur only once
    single_occurrence_org_parents = org_parents_single_occurrence['org_parent'].tolist()

    # Filter the original DataFrame to keep rows where 'org_parent' occurs only once
    org_parents_unique = orgs_df[orgs_df['org_parent'].isin(single_occurrence_org_parents)]

    print("\nOrgs which don't seem to be subsidiaries ...\n")
    display(org_parents_unique)

    single_entities = list(org_parents_unique[provider_col].unique())

    # Remove 'hdx' from single_entities, not good for testing as it's the folks that made HXL! Also some monolithic orgs with very similar data
    single_entities = [x for x in single_entities if not x in ['hdx','ourairports']]

    single_entities.sort()

    # Sample single-subsid orgs
    sample_size = int(len(single_entities)*test_size) - 1
    np.random.seed(42)
    X_test_orgs = np.random.choice(single_entities, sample_size)
    X_train_orgs = list(set(all_orgs)-set(X_test_orgs))

    print(f"Train orgs: {X_train_orgs}")
    print(f"Test orgs: {X_test_orgs}")

    # Extract column rows for datasets in X_train_datasets
    X_train = column_data[column_data[provider_col].isin(X_train_orgs)]

    # Extract column rows for datasets in X_test_datasets
    X_test = column_data[~column_data[provider_col].isin(X_train_orgs)]

    train_orgs = X_train[provider_col].unique()
    train_orgs.sort()
    test_orgs = X_test[provider_col].unique()
    test_orgs.sort()

    print(f"\nTrain orgs: {train_orgs}")
    print(f"\nTest orgs: {test_orgs}")

    print(f"\nTrain column data: {X_train.shape}")
    print(f"Test column data: {X_test.shape}")

    return X_train, X_test


X_train, X_test = split_data(data, 'Data provider', test_size=0.2, random_state=42)


Orgs which don't seem to be subsidiaries ...



,Data provider,count,org_parent
1,immap,221,immap
2,hdx,139,hdx
5,hera-humanitarian-emergency-response-africa,100,hera
11,redhum,69,redhum
13,fieldsdata,53,fieldsdata
17,ifrc,49,ifrc
20,wfp,41,wfp
25,brcmapsteam,31,brcmapsteam
28,dhs,28,dhs
29,standby-task-force,28,standby


Train orgs: ['ocha-nepal', 'ocha-rosc', 'cpaor', 'ocha-yemen', 'infoculture', 'libya-ingo-forum', 'education-cluster-yemen', 'ocha-cameroon', 'brcmapsteam', 'ocha-south-sudan', 'ocha-rowca', 'cred', 'international-displacement-monitoring-centre-idmc', 'unodc', 'crs-waro', 'hdx', 'ocha-mozambique-hat', 'world-bank-group', 'reliefweb', 'insecurity-insight', 'hera-humanitarian-emergency-response-africa', 'blavatnik-school-of-government-university-of-oxford', 'ocha-eritrea', 'cbes', 'ocha-sudan', 'ocha-ukraine', 'ocha-dr-congo', 'unicef-data', 'ocha-indonesia', 'sadc_rvaa', 'qcri', 'ocha-nigeria', 'undp-human-development-reports-office', 'soswcaf', 'hrrp-nepal', 'hxl', 'ocha-burkina', 'eth-zurich-weather-and-climate-risks', 'health-cluster', 'rcpwca', 'ifrc', 'ocha-somalia', 'moving-energy-initiative', 'ourairports', 'srsgcc', 'cirrolytix', 'unicef-esaro', 'ocha-ds', 'ocha-rosea', 'dalberg', 'cesvi', 'ocha-fts', 'ocha-niger', 'unhcr-afghanistan', 'unrwa-for-palestine-refugees-in-the-near-e

### Create prompt files

In [26]:
def create_prompt_file(X_train, prompt_col, filename):
    """
    Create a prompt file from a DataFrame.

    Args:
        X_train (pd.DataFrame): The DataFrame containing the prompts.
        prompt_col (str): The name of the column containing the prompts.
        filename (str): The name of the file to write the prompts to.
    """

    with open(filename, 'w') as f:
        for index, row in X_train.iterrows():
            f.write(row[prompt_col] + "\n")

    print(f"Prompts written to {filename}")

def generate_completion_prompt(dataset_name, resource_name, column_name, excerpt, hxl_tag=None, add_response=True):
    """
    Generate a completion (eg Davinci) fine-tuning prompt for HXL tags given dataset, resource, column information.

    Parameters:
    - dataset_name (str): Name of the dataset.
    - resource_name (str): Name of the resource.
    - column_name (str): Name of the column.
    - excerpt (str): Examples or excerpt of the column.
    - hxl_tag (str, optional): HXL tags for the column. Default is None.
    - add_response (bool, optional): Whether to include the response in the prompt. Default is True.

    Returns:
    - dict: A dictionary containing the prompt and optional completion/response.
    """
    resource_name = resource_name.replace(f"{LOCAL_DATA_DIR}/",'')
    #column_details = f"dataset_name:'{dataset_name}'; resource_name='{resource_name}'; column_name:{column_name}; examples:{excerpt}"
    column_details = f"resource_name='{resource_name}'; column_name:'{column_name}'; examples:{excerpt}"

    prompt = {"prompt": f"What are the HXL tags for a column with these attributes? {column_details}"}
    if add_response:
        prompt["completion"] = f"{hxl_tag}"
    return prompt

def generate_chat_prompt(dataset_name, resource_name, column_name, excerpt, hxl_tag=None, add_response=True):
    """
    Generate a chat (eg for GPT-3.5-Turbo) fine tuning prompt for HXL tags given dataset, resource, column information.

    Parameters:
    - dataset_name (str): Name of the dataset.
    - resource_name (str): Name of the resource.
    - column_name (str): Name of the column.
    - excerpt (str): Examples or excerpt of the column.
    - hxl_tag (str, optional): HXL tags for the column. Default is None.
    - add_response (bool, optional): Whether to include the response in the prompt. Default is True.

    Returns:
    - dict: A dictionary containing the prompt and optional completion/response.
    """

    system_message = """
        You are an assistant that replies with HXL tags and attributes"
    """

    resource_name = resource_name.replace(f"{LOCAL_DATA_DIR}/",'')
    #column_details = f"dataset_name:'{dataset_name}'; resource_name='{resource_name}'; column_name:{column_name}; examples:{excerpt}"
    column_details = f"resource_name='{resource_name}'; column_name:'{column_name}'; examples:{excerpt}"

    user_prompt = f"What are the HXL tags and attributes for a column with these details? {column_details}"

    prompt = {
        "messages": [
            {"role": "system", "content": system_message}, 
            {"role": "user", "content": user_prompt}, 
        ]
    }

    if add_response:
        prompt["messages"].append({"role": "assistant", "content": hxl_tag})

    #prompt = json.dumps(prompt)

    return prompt

def generate_prompts(df, 
                     heading_col='Text header', 
                     resource_name_col='File', \
                     tag_col='Hashtag with Attributes', \
                     excerpt_col='Data excerpt', \
                     hxl_tag=False,
                     prompt_type='completion'):
    """
    Generate a set of prompts for HXL tags from a DataFrame.

    Parameters:
    - df (DataFrame): Input DataFrame containing dataset, resource, column information.
    - hxl_tag (bool, optional): Whether to include HXL tags in the prompts. Default is False.
    - completion_prompt (string): One of 'completion', 'chat'

    Returns:
    - str: A string containing JSON-formatted prompts.
    """
    allowed_prompt_types =  ['completion', 'chat']
    if prompt_type not in allowed_prompt_types:
        print(f"Prompt type {prompt_type} is not one of {allowed_prompt_types}!")
        sys.exit()

    prompts = []
    for index, row in df.iterrows():
        if prompt_type == 'completion':
            prompt = generate_completion_prompt('',  # Dataset name
                                    row[resource_name_col], \
                                    row[heading_col], \
                                    row[excerpt_col], \
                                    hxl_tag=row[tag_col], \
                                    add_response=True)
        elif prompt_type == 'chat':
            prompt = generate_chat_prompt('',  # Dataset name
                        row[resource_name_col], \
                        row[heading_col], \
                        row[excerpt_col], \
                        hxl_tag=row[tag_col], \
                        add_response=True)

        prompts.append(prompt)
    return prompts

def save_prompts(prompts, filename):
    
    with open(filename, 'w') as f:
        for prompt in prompts:
            f.write(json.dumps(prompt) + "\n")

    print(f"Prompts written to {filename}")

# Prompt for use with davinci-002
#completion_prompts = generate_prompts(X_train, hxl_tag=False, prompt_type='completion')
#save_prompts(completion_prompts, "./data/hxl_completion_prompts.jsonl")


# Prompt for use with GPT-3.5-Turbo or GPT-4o-mini
chat_prompts = generate_prompts(X_train, hxl_tag=True, prompt_type='chat')
save_prompts(chat_prompts, "./data/hxl_chat_prompts.jsonl")


Prompts written to ./data/hxl_completion_prompts.jsonl
Prompts written to ./data/hxl_chat_prompts.jsonl


## Model Tuning Open AI

In [36]:
def fine_tune_model(train_file, model_name="gpt-4o-mini"):
    """
    Fine-tune an OpenAI model using training data.

    Args:
        prompt_file (str): The file containing the prompts to use for fine-tuning.
        model_name (str): The name of the model to fine-tune. Default is "davinci-002".

    Returns:
        str: The ID of the fine-tuned model.
    """

    client = OpenAI(
        api_key=os.getenv("OPENAI_API_KEY")
    )

    # Create a file on OpenAI for fine-tuning
    file = client.files.create(
        file=open(train_file, "rb"),
        purpose="fine-tune"
    )
    file_id = file.id
    print(f"Uploaded training file with ID: {file_id}")

    # Start the fine-tuning job
    ft = client.fine_tuning.jobs.create(
        training_file=file_id,
        model=model_name
    )
    ft_id = ft.id
    print(f"Fine-tuning job started with ID: {ft_id}")

    # Monitor the status of the fine-tuning job
    ft_result = client.fine_tuning.jobs.retrieve(ft_id)
    while ft_result.status != 'succeeded':
        print(f"Current status: {ft_result.status}")
        time.sleep(120)  # Wait for 60 seconds before checking again
        ft_result = client.fine_tuning.jobs.retrieve(ft_id)
        if 'failed' in ft_result.status.lower():
            sys.exit()

    print(f"Fine-tuning job {ft_id} succeeded!")

    # Retrieve the fine-tuned model
    fine_tuned_model = ft_result.fine_tuned_model
    print(f"Fine-tuned model: {fine_tuned_model}")

    return fine_tuned_model

In [37]:
fine_tune_model('./data/hxl_chat_prompts.jsonl', model_name="gpt-4o-mini-2024-07-18-alpha")

Uploaded training file with ID: file-GXG4Mo5hjlTMBQJZk38fNRFH
Fine-tuning job started with ID: ftjob-qRoWHtzzYA4yZqNZh0OGGkyd
Current status: validating_files
Current status: validating_files
Current status: queued
Current status: queued
Current status: queued
Current status: queued
Current status: queued
